In [ ]:
kubectl create namespace httpbin2
kubectl label namespace httpbin2 istio.io/dataplane-mode=ambient
kubectl label namespace httpbin2 istio.io/use-waypoint=waypoint
kubectl apply -f https://raw.githubusercontent.com/istio/istio/refs/heads/master/samples/sleep/sleep.yaml -n httpbin2
kubectl apply -f https://raw.githubusercontent.com/istio/istio/refs/heads/master/samples/httpbin/httpbin.yaml -n httpbin2

In [ ]:
kubectl apply -f- <<EOF
apiVersion: gateway.networking.k8s.io/v1
kind: Gateway
metadata:
  name: waypoint
  namespace: httpbin2
spec:
  gatewayClassName: agentgateway-enterprise-waypoint
  infrastructure:
    parametersRef:
      group: gateway.kgateway.dev
      kind: GatewayParameters
      name: agentgateway
  listeners:
  - allowedRoutes:
      namespaces:
        from: Same
    name: mesh
    port: 15008
    protocol: HBONE
---
apiVersion: gateway.kgateway.dev/v1alpha1
kind: GatewayParameters
metadata:
  name: agentgateway
  namespace: httpbin2
spec:
  kube:
    agentgateway:
      customConfigMapName: agentgateway-config
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: agentgateway-config
  namespace: httpbin2
data:
  config.yaml: |-
    config:
      logging:
        format: json
        fields:
          add:
            request.body: string(request.body)
            response.body: string(response.body)
EOF

In [ ]:
# gloo-gateway image: howardjohn/gloo-gateway-controller:1761079492
# kubectl set env deployment/waypoint CLUSTER_ID=cluster1

In [29]:
kubectl exec deploy/sleep -n httpbin2 -- curl -i http://httpbin:8000/headers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0HTTP/1.1 200 OK
100   146  100   146    0     0  13858      0 --:--:-- --:--:-- --:--:-- 14600
access-control-allow-credentials: true
access-control-allow-origin: *
content-type: application/json; charset=utf-8
date: Thu, 23 Oct 2025 18:02:24 GMT
content-length: 146

{
  "headers": {
    "Accept": [
      "*/*"
    ],
    "Host": [
      "httpbin:8000"
    ],
    "User-Agent": [
      "curl/8.16.0"
    ]
  }
}


In [30]:
kubectl logs deploy/waypoint -n httpbin2 --tail=50

{"level":"info","time":"2025-10-23T17:10:48.052056Z","scope":"agent_xds::client","message":"received response","type_url":"type.googleapis.com/agentgateway.dev.workload.Address","size":3,"removes":0,"xds":{"id":1}}
{"level":"info","time":"2025-10-23T17:10:51.695151Z","scope":"agent_xds::client","message":"received response","type_url":"type.googleapis.com/agentgateway.dev.workload.Address","size":0,"removes":1,"xds":{"id":1}}
{"level":"info","time":"2025-10-23T17:10:52.165187Z","scope":"agent_xds::client","message":"received response","type_url":"type.googleapis.com/agentgateway.dev.workload.Address","size":0,"removes":1,"xds":{"id":1}}
{"level":"info","time":"2025-10-23T17:10:52.455068Z","scope":"agent_xds::client","message":"received response","type_url":"type.googleapis.com/agentgateway.dev.workload.Address","size":0,"removes":1,"xds":{"id":1}}
{"level":"info","time":"2025-10-23T17:10:55.026338Z","scope":"agent_xds::client","message":"received response","type_url":"type.googleapis.c